# ChatAnthropic

This notebook covers how to get started with Anthropic chat models.

## Setup

For setup instructions, please see the Installation and Environment Setup sections of the [Anthropic Platform page](/docs/integrations/platforms/anthropic.mdx).

In [ ]:
%pip install -qU langchain-anthropic

## Environment Setup

We'll need to get a [Anthropic](https://console.anthropic.com/settings/keys) and set the `ANTHROPIC_API_KEY` environment variable:

In [ ]:
import os
from getpass import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass()

The code provided assumes that your ANTHROPIC_API_KEY is set in your environment variables. If you would like to manually specify your API key and also choose a different model, you can use the following code:
```python
chat = ChatAnthropic(temperature=0, anthropic_api_key="YOUR_API_KEY", model_name="claude-instant-1.2")

```
Please note that the default model is "claude-2," and you can check the available models at [here](https://docs.anthropic.com/claude/reference/selecting-a-model).

In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

chat = ChatAnthropic(temperature=0, model_name="claude-2")

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

AIMessage(content=' 저는 파이썬을 좋아합니다.')

## `ChatAnthropic` also supports async and streaming functionality:

In [4]:
chat = ChatAnthropic(temperature=0, model_name="claude-2")
prompt = ChatPromptTemplate.from_messages([("human", "Tell me a joke about {topic}")])
chain = prompt | chat
await chain.ainvoke({"topic": "bear"})

AIMessage(content=" Why don't bears like fast food? Because they can't catch it!")

In [5]:
chat = ChatAnthropic(temperature=0.3, model_name="claude-2")
prompt = ChatPromptTemplate.from_messages(
    [("human", "Give me a list of famous tourist attractions in Japan")]
)
chain = prompt | chat
for chunk in chain.stream({}):
    print(chunk.content, end="", flush=True)

 Here are some of the most famous tourist attractions in Japan:

- Tokyo - Tokyo Tower, Tokyo Skytree, Imperial Palace, Sensoji Temple, Meiji Shrine, Shibuya Crossing

- Kyoto - Kinkakuji (Golden Pavilion), Fushimi Inari Shrine, Kiyomizu-dera Temple, Arashiyama Bamboo Grove, Gion Geisha District

- Osaka - Osaka Castle, Dotonbori, Universal Studios Japan, Osaka Aquarium Kaiyukan 

- Hiroshima - Hiroshima Peace Memorial Park and Museum, Itsukushima Shrine (Miyajima Island)

- Mount Fuji - Iconic and famous mountain, popular for hiking and viewing from places like Hakone and Kawaguchiko Lake

- Himeji - Himeji Castle, one of Japan's most impressive feudal castles

- Nara - Todaiji Temple, Nara Park with its bowing deer, Horyuji Temple with some of world's oldest wooden structures  

- Nikko - Elaborate shrines and temples nestled around Nikko National Park

- Sapporo - Snow